In [5]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook
import re
import nltk
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer

from keras.layers import Dense, Activation, Dropout, Flatten, Input
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.models import Model, Sequential
from keras.layers.recurrent import LSTM

import tensorflow as tf
from keras import backend as K
from keras.layers.merge import concatenate
from keras.utils import plot_model

from subprocess import check_output
dir = '/Users/xinwang/ai/dataset/kaggle/DonorsChoose/'
print(check_output(['ls', dir]).decode('utf8'))


train = pd.read_csv(dir + 'train.csv', low_memory=False, index_col='id')
test = pd.read_csv(dir + 'test.csv', low_memory=False, index_col='id')
res = pd.read_csv(dir + 'resources.csv',low_memory=False, index_col='id')

train['is_train'] = 1
test['is_train'] = 0

df = pd.concat([train, test], axis=0)

df.head()

resources.csv
sample_submission.csv
test.csv
train.csv
train_1.csv
train_2.csv



/Users/xinwang/ai/lib/python3.6/site-packages/ipykernel_launcher.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,is_train,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,project_submitted_datetime,project_title,school_state,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix
id,,,,,,,,,,,,,,,,
p036502,1,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,Grades PreK-2,1.0,My students need 6 Ipod Nano's to create and d...,Literacy & Language,Literacy,2016-11-18 14:45:59,Super Sight Word Centers,NV,484aaf11257089a66cfedc9461c6bd0a,26,Ms.
p039565,1,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,Grades 3-5,0.0,My students need matching shirts to wear for d...,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",2017-04-26 15:57:28,Keep Calm and Dance On,GA,df72a3ba8089423fa8a94be88060f6ed,1,Mrs.
p233823,1,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,Grades 3-5,1.0,My students need the 3doodler. We are an SEM s...,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",2017-01-01 22:57:44,Lets 3Doodle to Learn,UT,a9b876a9252e08a55e3d894150f75ba3,5,Ms.
p185307,1,My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,Grades 3-5,0.0,My students need balls and other activity equi...,Health & Sports,Health & Wellness,2016-08-12 15:42:11,"\""Kid Inspired\"" Equipment to Increase Activit...",NC,525fdbb6ec7f538a48beebaa0a51b24f,16,Mr.
p013780,1,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,Grades 6-8,1.0,My students need a water filtration system for...,Health & Sports,Health & Wellness,2016-08-06 09:09:11,We need clean water for our culinary arts class!,CA,a63b5547a7239eae4c1872670848e61a,42,Mr.


In [12]:
sum_res = pd.pivot_table(res, index=res.index, aggfunc='sum', values=['price','quantity'])
print(sum_res.head())

mean_res = pd.pivot_table(res, index=res.index, aggfunc='mean', values=['price','quantity'])
print(mean_res.head())

median_res = pd.pivot_table(res, index=res.index, aggfunc='median', values=['price','quantity'])
print(median_res[0:5])


df = pd.merge(df, sum_res, left_index=True, right_index=True)
df = pd.merge(df, mean_res, left_index=True, right_index=True, suffixes = ('_sum',''))
df = pd.merge(df, median_res, left_index=True, right_index=True, suffixes=('_mean','_median'))

df.head()

           price  quantity
id                        
p000001   459.56         7
p000002   515.89        21
p000003   298.97         4
p000004  1113.69        98
p000005   485.99         8
              price  quantity
id                           
p000001  114.890000  1.750000
p000002   36.849286  1.500000
p000003   74.742500  1.000000
p000004   11.723053  1.031579
p000005  121.497500  2.000000
          price  quantity
id                       
p000001  87.245       2.0
p000002  29.990       1.0
p000003  44.990       1.0
p000004   7.260       1.0
p000005  54.080       2.0


,is_train,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_grade_category,project_is_approved,project_resource_summary,project_subject_categories,project_subject_subcategories,...,school_state,teacher_id,teacher_number_of_previously_posted_projects,teacher_prefix,price_sum,quantity_sum,price_mean,quantity_mean,price_median,quantity_median
id,,,,,,,,,,,,,,,,,,,,,
p036502,1,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,Grades PreK-2,1.0,My students need 6 Ipod Nano's to create and d...,Literacy & Language,Literacy,...,NV,484aaf11257089a66cfedc9461c6bd0a,26,Ms.,299.98,6,149.990,3.0,149.99,3.0
p039565,1,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,Grades 3-5,0.0,My students need matching shirts to wear for d...,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",...,GA,df72a3ba8089423fa8a94be88060f6ed,1,Mrs.,20.00,20,20.000,20.0,20.00,20.0
p233823,1,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,Grades 3-5,1.0,My students need the 3doodler. We are an SEM s...,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",...,UT,a9b876a9252e08a55e3d894150f75ba3,5,Ms.,469.99,1,469.990,1.0,469.99,1.0
p185307,1,My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,Grades 3-5,0.0,My students need balls and other activity equi...,Health & Sports,Health & Wellness,...,NC,525fdbb6ec7f538a48beebaa0a51b24f,16,Mr.,684.47,5,136.894,1.0,126.87,1.0
p013780,1,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,Grades 6-8,1.0,My students need a water filtration system for...,Health & Sports,Health & Wellness,...,CA,a63b5547a7239eae4c1872670848e61a,42,Mr.,355.50,2,355.500,2.0,355.50,2.0


In [13]:
print(df.columns)

Index(['is_train', 'project_essay_1', 'project_essay_2', 'project_essay_3',
       'project_essay_4', 'project_grade_category', 'project_is_approved',
       'project_resource_summary', 'project_subject_categories',
       'project_subject_subcategories', 'project_submitted_datetime',
       'project_title', 'school_state', 'teacher_id',
       'teacher_number_of_previously_posted_projects', 'teacher_prefix',
       'price_sum', 'quantity_sum', 'price_mean', 'quantity_mean',
       'price_median', 'quantity_median'],
      dtype='object')


In [18]:
cat_feature = ['school_state','teacher_prefix','project_subject_categories','project_subject_subcategories','project_grade_category']
text_feature = ['project_title','project_resource_summary','project_essay_1','project_essay_2','project_essay_3','project_essay_4']
real_feature = ['teacher_number_of_previously_posted_projects','price_sum','quantity_sum','price_mean','quantity_mean','price_median','quantity_median']
target = 'project_is_approved'


for i in cat_feature:
    df[i] = pd.factorize(df[i])[0]

split_size = 182080
trn_cat = df[cat_feature].values[:split_size]
tst_cat = df[cat_feature].values[split_size:]

SS = StandardScaler()
df_scale = SS.fit_transform(df[real_feature])

trn_real = df_scale[:split_size]
tst_real = df_scale[split_size:]

df_text = df[text_feature].fillna(' ')
df_text['full_text'] = ' '

for f in text_feature:
    df_text['full_text'] = df_text['full_text'] + df_text[f]

    
stemmer = SnowballStemmer('english')

def clean(text):
    return re.sub('[!@#$:]', '', ' '.join(re.findall('\w{3,}', str(text).lower())))

def stem(text):
    return ' '.join([stemmer.stem(w) for w in text.split()])

df_text['full_text'] = df_text['full_text'].apply(lambda x: clean(x))

pd.set_option('max_colwidth', 80000)
df_text['full_text'][0:2]
pd.set_option('max_colwidth', 80)

id
p036502                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 super sight word centersmy students need ipod nano create and differentiated and engaging way practice sight words during literacy station most kindergarten students come from low income households and are considered risk these kids walk school alongside their parents and most have never been further

In [35]:
max_words = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_text['full_text'])

trn_text = tokenizer.texts_to_matrix(df_text['full_text'][:split_size], mode='binary')
tst_text = tokenizer.texts_to_matrix(df_text['full_text'][split_size:], mode='binary')

y = df[target].values[:split_size]

print('trn_text.shape', trn_text.shape)
trn_text[0:3]

trn_text.shape (182080, 5000)


array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [36]:
len_cat = trn_cat.shape[1]
len_real = trn_real.shape[1]
len_text = trn_text.shape[1]

size_embedding = 5000

inputs1 = Input(shape=(len_cat,))
dense_cat_1 = Dense(256, activation='relu')(inputs1)
dense_cat_2 = Dense(128, activation='relu')(dense_cat_1)
dense_cat_3 = Dense(64, activation='relu')(dense_cat_2)
dense_cat_4 = Dense(32, activation='relu')(dense_cat_3)

flat1 = Dense(32, activation='relu')(dense_cat_4)


inputs2 = Input(shape=(len_real,))
dense_real_1 = Dense(256, activation='relu')(inputs2)
dense_real_2 = Dense(128, activation='relu')(dense_real_1)
dense_real_3 = Dense(64, activation='relu')(dense_real_2)
dense_real_4 = Dense(32, activation='relu')(dense_real_3)
flat2 = Dense(32, activation='relu')(dense_real_4)


inputs3 = Input(shape=(len_text,))
embedding3 = Embedding(size_embedding, 36)(inputs3)
conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.1)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)

merged = concatenate([flat1, flat2, flat3])

dense1 = Dense(200, activation='relu')(merged)
dense2 = Dense(20, activation='relu')(dense1)
outputs = Dense(1, activation='sigmoid')(dense2)

model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 5000)         0                                            
__________________________________________________________________________________________________
dense_68 (Dense)                (None, 256)          1536        input_16[0][0]                   
__________________________________________________________________________________________________
dense_73 (

In [38]:
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true, y_pred, k) for k in np.linspace(0,1,1000)], axis=0)
    pfas = tf.stack([binary_PFA(y_true, y_pred, k) for k in np.linspace(0,1,1000)], axis=0)
    
    pfas = tf.concat([tf.ones((1,)), pfas], axis=0)
    binSize = -(pfas[1:]-pfas[:-1])
    
    s = ptas * binSize
    
    return K.sum(s, axis=0)


def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    N = K.sum(1-y_true)
    FP = K.sum(y_pred-y_pred*y_true)
    
    return FP/N


def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    P = K.sum(y_true)
    
    TP = K.sum(y_pred * y_true)
    
    return TP/P


model.compile(loss='binary_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy', auc])


batch_size = 1000
model.fit([trn_cat, trn_real, trn_text], y, batch_size=batch_size, epochs=3, validation_split=0.2)

Train on 145664 samples, validate on 36416 samples
Epoch 1/3
145664/145664 [==============================] - 1807s 12ms/step - loss: 0.3834 - acc: 0.8487 - auc: 0.7291 - val_loss: 0.3755 - val_acc: 0.8504 - val_auc: 0.7399
Epoch 2/3
145664/145664 [==============================] - 1877s 13ms/step - loss: 0.3683 - acc: 0.8537 - auc: 0.7589 - val_loss: 0.3748 - val_acc: 0.8514 - val_auc: 0.7413
Epoch 3/3
145664/145664 [==============================] - 1903s 13ms/step - loss: 0.3586 - acc: 0.8560 - auc: 0.7762 - val_loss: 0.3746 - val_acc: 0.8518 - val_auc: 0.7432


In [39]:
submit = model.predict([tst_cat, tst_real, tst_text], batch_size=batch_size, verbose=1)

submission = pd.read_csv(dir + 'sample_submission.csv')
submission['project_is_approved'] = submit
submission.to_csv('mi_nn.csv', index=False)

78035/78035 [==============================] - 435s 6ms/step
